In [4]:
from system.mightypix_new_pixel import Run2021
import numpy as np
from run2021_daq.system.mio3 import MIO3
import run2021_daq.analysis.interpreter_clean as interpreter
import run2021_daq.analysis.analysis_utils as au






In [5]:
daq = MIO3(conf="system/mio3.yaml")
daq.init()

chip = Run2021(daq, conf=daq.configuration)
chip.init()

start_column=0
stop_column=29
start_row=0
stop_row=124

chip.masks['enable'][start_column:stop_column, start_row:stop_row] = True
chip.masks['injection'][start_column:stop_column, start_row:stop_row] = True

chip.masks.apply_disable_mask()
chip.masks.update()

2025-05-14 10:18:23,065 [basil.HL.GPAC_MightyPix] - INFO    General Purpose Analog Card (GPAC): skip reading calibration parameters from EEPROM
2025-05-14 10:18:23,075 [Run2021 - ChipW00R00] - INFO    Setting Vminus to 0.8
2025-05-14 10:18:23,079 [Run2021 - ChipW00R00] - INFO    Setting TH1 to 1.2
2025-05-14 10:18:23,082 [Run2021 - ChipW00R00] - INFO    Setting BL to 0.94
2025-05-14 10:18:23,085 [Run2021 - ChipW00R00] - INFO    Setting VDDR to 1.8
2025-05-14 10:18:23,086 [basil.HL.GPAC_MightyPix] - WARNING VOUTD DAC value reached maximum value
2025-05-14 10:18:23,090 [Run2021 - ChipW00R00] - INFO    Setting TH2 to 1.8
2025-05-14 10:18:23,090 [basil.HL.GPAC_MightyPix] - WARNING VOUTE DAC value reached maximum value
2025-05-14 10:18:23,093 [Run2021 - ChipW00R00] - INFO    Setting TH3 to 1.2
2025-05-14 10:18:23,099 [Run2021 - ChipW00R00] - INFO    Setting ThPix to 255 : 1.7850000000000001V
2025-05-14 10:18:23,100 [Run2021 - ChipW00R00] - INFO    Setting BlPix to 133 : 0.931V
2025-05-14 10

True

In [6]:
import sys
import os
import time
from datetime import datetime
from pyvisa import ResourceManager
import serial.tools.list_ports
from arduino_powersup import arduino, read_hw_mean, arduino_power_plot
sys.path.append("/home/lab/daq/power_supply_monitoring")
from hmp4040_interface import *


rm = ResourceManager()
address='ASRL/dev/ttyUSB0::INSTR'
hw = HMP4040(address,rm,"HO720",max_channels=4)
ports = serial.tools.list_ports.comports()


def write_csv(data, filename):
    with open(filename, 'a') as file:
        file.write(f'{data}\n')





dac_names = ['VN', 'IPLoad', 'VNFoll','INFB', 'IBLRes', 'VNComp', 'IPDAC',  'VNBiasRec', 'IPBiasRec', 'VNDel'] # DAC names to be used in the test
pin_mapping = { # maps the arduino pins to the chips pin descriptions. (code 0-7 -> ard left A0-A7), (code 8-15 -> ard right A0-A7)
    0: 'IPDAC',
    1: "VNDel",
    2: "VNBiasRec",
    3: "IPBiasRec",
    8: "IBLRes",
    9: "VN",
    10: "INFB",
    11: "VNFoll",
    12: "IPLoad",
    13: "VNComp"
}

BestrahlungsNummer = '50kRad'
chip_id = '001'
path = f'output_data/tid/Max/{BestrahlungsNummer}/90V/dac_power/'
infostring = f'temp = 20°C \n HV = -90V'


for dac_name in dac_names:
    directory = f'{path}{dac_name}/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    header = f"DAC_name, DAC_set_value, DAC_measured, vddd_volt, vddd_curr, vddd_powr, vdda_volt, vdda_curr, vdda_powr, timestamp"
    filename = f"{directory}/{dac_name}_{BestrahlungsNummer}_chip{chip_id}_{timestamp}.csv"
    write_csv(header, filename)
    print(f"Scanning {dac_name} DAC values...")

    for dac_value in range(64):
        chip.daq['CONF_SR'][dac_name] = dac_value
        chip.daq['CONF_SR'].write()
        time.sleep(0.5)

        ar_values = arduino(24, pin_mapping) # Nummer = Anzahl an Messungen aus dem der mittelwert gebildet wird, pin_mapping = Pin Mapping (0-15) 
        hw_values = read_hw_mean(hw, channel1 = 1, channel2 = 2, n = 24) # hw braucht er zum auslesen. channel 1 und 2 sind die CH Kanäle des Netzteils. n ist Anzahl an Messungen, aus denen der Mittelwert gebildet wird
        
        data = f"{dac_name}, {dac_value}, {ar_values[dac_name][0]}, {hw_values['vddd_volt']}, {hw_values['vddd_curr']}, {hw_values['vddd_powr']}, {hw_values['vdda_volt']}, {hw_values['vdda_curr']}, {hw_values['vdda_powr']}, {timestamp}"
        write_csv(data, filename)
    chip.init()
    arduino_power_plot(path, path, infostring, save=False)
    time.sleep(2)

arduino_power_plot(path, path, infostring, save=True)

    

ASRL/dev/ttyUSB0::INSTR
SerialInstrument at ASRL/dev/ttyUSB0::INSTR
HAMEG,HMP4040,105763,HW50020001/SW2.51

Scanning VN DAC values...


2025-05-14 10:22:59,492 [Run2021 - ChipW00R00] - INFO    Setting Vminus to 0.8
2025-05-14 10:22:59,496 [Run2021 - ChipW00R00] - INFO    Setting TH1 to 1.2
2025-05-14 10:22:59,499 [Run2021 - ChipW00R00] - INFO    Setting BL to 0.94
2025-05-14 10:22:59,502 [Run2021 - ChipW00R00] - INFO    Setting VDDR to 1.8
2025-05-14 10:22:59,502 [basil.HL.GPAC_MightyPix] - WARNING VOUTD DAC value reached maximum value
2025-05-14 10:22:59,506 [Run2021 - ChipW00R00] - INFO    Setting TH2 to 1.8
2025-05-14 10:22:59,506 [basil.HL.GPAC_MightyPix] - WARNING VOUTE DAC value reached maximum value
2025-05-14 10:22:59,510 [Run2021 - ChipW00R00] - INFO    Setting TH3 to 1.2
2025-05-14 10:22:59,516 [Run2021 - ChipW00R00] - INFO    Setting ThPix to 255 : 1.7850000000000001V
2025-05-14 10:22:59,517 [Run2021 - ChipW00R00] - INFO    Setting BlPix to 133 : 0.931V
2025-05-14 10:22:59,519 [Run2021 - ChipW00R00] - INFO    Setting ThPix2 to 160 : 1.12V
2025-05-14 10:22:59,520 [Run2021 - ChipW00R00] - INFO    Setting VCASC

Scanning IPLoad DAC values...


2025-05-14 10:26:42,875 [Run2021 - ChipW00R00] - INFO    Setting Vminus to 0.8
2025-05-14 10:26:42,879 [Run2021 - ChipW00R00] - INFO    Setting TH1 to 1.2
2025-05-14 10:26:42,883 [Run2021 - ChipW00R00] - INFO    Setting BL to 0.94
2025-05-14 10:26:42,886 [Run2021 - ChipW00R00] - INFO    Setting VDDR to 1.8
2025-05-14 10:26:42,886 [basil.HL.GPAC_MightyPix] - WARNING VOUTD DAC value reached maximum value
2025-05-14 10:26:42,890 [Run2021 - ChipW00R00] - INFO    Setting TH2 to 1.8
2025-05-14 10:26:42,890 [basil.HL.GPAC_MightyPix] - WARNING VOUTE DAC value reached maximum value
2025-05-14 10:26:42,894 [Run2021 - ChipW00R00] - INFO    Setting TH3 to 1.2
2025-05-14 10:26:42,900 [Run2021 - ChipW00R00] - INFO    Setting ThPix to 255 : 1.7850000000000001V
2025-05-14 10:26:42,902 [Run2021 - ChipW00R00] - INFO    Setting BlPix to 133 : 0.931V
2025-05-14 10:26:42,903 [Run2021 - ChipW00R00] - INFO    Setting ThPix2 to 160 : 1.12V
2025-05-14 10:26:42,904 [Run2021 - ChipW00R00] - INFO    Setting VCASC

Scanning VNFoll DAC values...


...

2025-05-14 10:49:07,461 [Run2021 - ChipW00R00] - INFO    Setting Vminus to 0.8
2025-05-14 10:49:07,464 [Run2021 - ChipW00R00] - INFO    Setting TH1 to 1.2
2025-05-14 10:49:07,468 [Run2021 - ChipW00R00] - INFO    Setting BL to 0.94
2025-05-14 10:49:07,471 [Run2021 - ChipW00R00] - INFO    Setting VDDR to 1.8
2025-05-14 10:49:07,472 [basil.HL.GPAC_MightyPix] - WARNING VOUTD DAC value reached maximum value
2025-05-14 10:49:07,475 [Run2021 - ChipW00R00] - INFO    Setting TH2 to 1.8
2025-05-14 10:49:07,475 [basil.HL.GPAC_MightyPix] - WARNING VOUTE DAC value reached maximum value
2025-05-14 10:49:07,479 [Run2021 - ChipW00R00] - INFO    Setting TH3 to 1.2
2025-05-14 10:49:07,485 [Run2021 - ChipW00R00] - INFO    Setting ThPix to 255 : 1.7850000000000001V
2025-05-14 10:49:07,487 [Run2021 - ChipW00R00] - INFO    Setting BlPix to 133 : 0.931V
2025-05-14 10:49:07,488 [Run2021 - ChipW00R00] - INFO    Setting ThPix2 to 160 : 1.12V
2025-05-14 10:49:07,489 [Run2021 - ChipW00R00] - INFO    Setting VCASC

Scanning IPBiasRec DAC values...


2025-05-14 10:52:53,260 [Run2021 - ChipW00R00] - INFO    Setting Vminus to 0.8
2025-05-14 10:52:53,264 [Run2021 - ChipW00R00] - INFO    Setting TH1 to 1.2
2025-05-14 10:52:53,267 [Run2021 - ChipW00R00] - INFO    Setting BL to 0.94
2025-05-14 10:52:53,270 [Run2021 - ChipW00R00] - INFO    Setting VDDR to 1.8
2025-05-14 10:52:53,270 [basil.HL.GPAC_MightyPix] - WARNING VOUTD DAC value reached maximum value
2025-05-14 10:52:53,274 [Run2021 - ChipW00R00] - INFO    Setting TH2 to 1.8
2025-05-14 10:52:53,274 [basil.HL.GPAC_MightyPix] - WARNING VOUTE DAC value reached maximum value
2025-05-14 10:52:53,277 [Run2021 - ChipW00R00] - INFO    Setting TH3 to 1.2
2025-05-14 10:52:53,282 [Run2021 - ChipW00R00] - INFO    Setting ThPix to 255 : 1.7850000000000001V
2025-05-14 10:52:53,284 [Run2021 - ChipW00R00] - INFO    Setting BlPix to 133 : 0.931V
2025-05-14 10:52:53,285 [Run2021 - ChipW00R00] - INFO    Setting ThPix2 to 160 : 1.12V
2025-05-14 10:52:53,286 [Run2021 - ChipW00R00] - INFO    Setting VCASC

Scanning VNDel DAC values...


2025-05-14 10:56:37,506 [Run2021 - ChipW00R00] - INFO    Setting Vminus to 0.8
2025-05-14 10:56:37,510 [Run2021 - ChipW00R00] - INFO    Setting TH1 to 1.2
2025-05-14 10:56:37,514 [Run2021 - ChipW00R00] - INFO    Setting BL to 0.94
2025-05-14 10:56:37,517 [Run2021 - ChipW00R00] - INFO    Setting VDDR to 1.8
2025-05-14 10:56:37,517 [basil.HL.GPAC_MightyPix] - WARNING VOUTD DAC value reached maximum value
2025-05-14 10:56:37,520 [Run2021 - ChipW00R00] - INFO    Setting TH2 to 1.8
2025-05-14 10:56:37,521 [basil.HL.GPAC_MightyPix] - WARNING VOUTE DAC value reached maximum value
2025-05-14 10:56:37,524 [Run2021 - ChipW00R00] - INFO    Setting TH3 to 1.2
2025-05-14 10:56:37,530 [Run2021 - ChipW00R00] - INFO    Setting ThPix to 255 : 1.7850000000000001V
2025-05-14 10:56:37,532 [Run2021 - ChipW00R00] - INFO    Setting BlPix to 133 : 0.931V
2025-05-14 10:56:37,533 [Run2021 - ChipW00R00] - INFO    Setting ThPix2 to 160 : 1.12V
2025-05-14 10:56:37,534 [Run2021 - ChipW00R00] - INFO    Setting VCASC